# Python Function API

- 部分示例代码，根据需要，做了一点修改，方便更好的调试。

## Reference

- https://duckdb.org/docs/stable/clients/python/function




In [ ]:
import duckdb
from duckdb.typing import *
from faker import Faker


# Connect to an in-memory database and create a function to generate random names.
con = duckdb.connect(":memory:")


def generate_random_name():
    fake = Faker()
    return fake.name()


con.create_function("random_name", generate_random_name, [], VARCHAR)

# res = duckdb.sql("SELECT random_name()").fetchall()
# print(res)

con.sql("SELECT random_name()").df()

,random_name()
0,Jacqueline Mejia


In [ ]:
import duckdb


#
# https://duckdb.org/docs/stable/clients/python/function#type-annotation
#
def my_function(x: int) -> str:
    return x


# 别名
duckdb.create_function("my_func", my_function)
print(duckdb.sql("SELECT my_func(42)"))

┌─────────────┐
│ my_func(42) │
│   varchar   │
├─────────────┤
│ 42          │
└─────────────┘



In [16]:
import duckdb
from duckdb.typing import *


def dont_intercept_null(x):
    return 5


# 避免重复注册导致报错
duckdb.remove_function("dont_intercept")

# 注册
duckdb.create_function("dont_intercept", dont_intercept_null, [BIGINT], BIGINT)

# res = duckdb.sql("SELECT dont_intercept(NULL)").fetchall()
# print(res)

duckdb.sql("SELECT dont_intercept(NULL)").df()

,dont_intercept(NULL)
0,<NA>


In [17]:
import duckdb
from duckdb.typing import *


def dont_intercept_null(x):
    return 5


# 避免重复注册导致报错
duckdb.remove_function("dont_intercept")

duckdb.create_function(
    "dont_intercept", dont_intercept_null, [BIGINT], BIGINT, null_handling="special"
)
res = duckdb.sql("SELECT dont_intercept(NULL)").fetchall()
print(res)

[(5,)]


In [ ]:
import duckdb
from duckdb.typing import *


def will_throw():
    raise ValueError("ERROR")


try:
    # 注册
    duckdb.create_function("throws", will_throw, [], BIGINT)
except Exception as e:
    duckdb.remove_function("throws")
    duckdb.create_function("throws", will_throw, [], BIGINT)

try:
    res = duckdb.sql("SELECT throws()").fetchall()
except duckdb.InvalidInputException as e:
    print(e)

duckdb.create_function(
    "doesnt_throw", will_throw, [], BIGINT, exception_handling="return_null"
)
res = duckdb.sql("SELECT doesnt_throw()").fetchall()
print(res)

In [20]:
import duckdb


def count() -> int:
    old = count.counter
    count.counter += 1
    return old


count.counter = 0


con = duckdb.connect()

con.create_function("my_counter", count, side_effects=False)

res = con.sql("SELECT my_counter() FROM range(10)").fetchall()
print(res)

[(0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,), (0,)]


In [21]:
con.remove_function("my_counter")

count.counter = 0

con.create_function("my_counter", count, side_effects=True)

res = con.sql("SELECT my_counter() FROM range(10)").fetchall()
print(res)

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]


In [36]:
import duckdb

from duckdb.typing import *
from faker import Faker


def random_date():
    fake = Faker()
    return fake.date_between()


try:
    duckdb.create_function("random_date", random_date, [], DATE, type="native")
except Exception as e:
    duckdb.remove_function("random_date")
    duckdb.create_function("random_date", random_date, [], DATE, type="native")


res = duckdb.sql("SELECT random_date()").fetchall()
print(res)

[(datetime.date(2018, 1, 18),)]
